<a href="https://colab.research.google.com/github/raki-rankawat/stm32/blob/main/VWW_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
import tarfile
import random
import shutil
from pathlib import Path
from urllib.request import urlretrieve

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# ----------------------------
# Auto Download + Prepare VWW (10k subset)
# ----------------------------
# VWW preprocessed dataset (96x96) download URL
vww_url = "https://www.silabs.com/public/files/github/machine_learning/benchmarks/datasets/vw_coco2014_96.tar.gz"

base_dir = Path("/content/vww_work")
archive_path = base_dir / "vw_coco2014_96.tar.gz"
extract_dir = base_dir / "extracted"
subset_dir = base_dir / "vww_10k"

# Subset config: 5k person + 5k non_person
n_per_class = 5000
val_ratio = 0.10

random.seed(41)
torch.manual_seed(41)

def download_vww():
    base_dir.mkdir(parents=True, exist_ok=True)

    if archive_path.exists() and archive_path.stat().st_size > 0:
        print("✅ VWW archive already downloaded")
        return

    print("⬇️ Downloading VWW archive...")
    urlretrieve(vww_url, archive_path)
    print("✅ Download complete:", archive_path)

def extract_vww():
    extract_dir.mkdir(parents=True, exist_ok=True)

    if any(extract_dir.iterdir()):
        print("✅ VWW already extracted")
        return

    print("📦 Extracting VWW archive...")
    with tarfile.open(archive_path, "r:gz") as tar:
        tar.extractall(extract_dir)
    print("✅ Extraction complete:", extract_dir)

def find_vww_root():
    # Find folder that contains BOTH person/ and non_person/
    for p in extract_dir.rglob("person"):
        if p.is_dir() and (p.parent / "non_person").is_dir():
            return p.parent
    raise RuntimeError("❌ Could not find 'person' and 'non_person' directories under extracted dataset")

def list_images(folder):
    exts = {".jpg", ".jpeg", ".png"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

def make_vww_subset(src_root):
    # Skip if subset already exists
    if (subset_dir / "train" / "person").is_dir() and (subset_dir / "val" / "non_person").is_dir():
        print("✅ VWW 10k subset already exists:", subset_dir)
        return

    for split in ["train", "val"]:
        for c in ["person", "non_person"]:
            (subset_dir / split / c).mkdir(parents=True, exist_ok=True)

    person_imgs = list_images(src_root / "person")
    nonperson_imgs = list_images(src_root / "non_person")

    if len(person_imgs) < n_per_class or len(nonperson_imgs) < n_per_class:
        raise ValueError(
            f"❌ Not enough images:\n"
            f"person: {len(person_imgs)} (need {n_per_class})\n"
            f"non_person: {len(nonperson_imgs)} (need {n_per_class})"
        )

    random.shuffle(person_imgs)
    random.shuffle(nonperson_imgs)

    person_sel = person_imgs[:n_per_class]
    nonperson_sel = nonperson_imgs[:n_per_class]

    def split_list(lst, val_ratio):
        n_val = int(len(lst) * val_ratio)
        return lst[n_val:], lst[:n_val]  # train, val

    p_train, p_val = split_list(person_sel, val_ratio)
    n_train, n_val = split_list(nonperson_sel, val_ratio)

    def copy_files(files, dst_dir):
        for f in files:
            dst = dst_dir / f.name
            # avoid rare collisions
            if dst.exists():
                dst = dst_dir / (f"{f.parent.name}_{f.name}")
            shutil.copy2(f, dst)

    print("🧩 Creating VWW 10k subset...")
    copy_files(p_train, subset_dir / "train" / "person")
    copy_files(p_val,   subset_dir / "val"   / "person")
    copy_files(n_train, subset_dir / "train" / "non_person")
    copy_files(n_val,   subset_dir / "val"   / "non_person")
    print("✅ VWW subset created at:", subset_dir)

download_vww()
extract_vww()
vww_root = find_vww_root()
print("✅ Found VWW root:", vww_root)
make_vww_subset(vww_root)

✅ VWW archive already downloaded
✅ VWW already extracted
✅ Found VWW root: /content/vww_work/extracted/vw_coco2014_96
✅ VWW 10k subset already exists: /content/vww_work/vww_10k


In [3]:
# ----------------------------
# Data Loaders (same style)
# ----------------------------
batch_size = 64
img_size = 96

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_data = datasets.ImageFolder(root=str(subset_dir / "train"), transform=train_transform)
test_data  = datasets.ImageFolder(root=str(subset_dir / "val"),   transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=batch_size, shuffle=False)

print("Class mapping:", train_data.class_to_idx)

Class mapping: {'non_person': 0, 'person': 1}


In [4]:
# ----------------------------
# CNN Model (same style)
# ----------------------------
class VWWConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        # For 96x96 with 4 pools: 96->48->24->12->6
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 2)   # VWW: person vs non_person

        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2) # 96 -> 48

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2) # 48 -> 24

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2) # 24 -> 12

        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2) # 12 -> 6

        x = x.view(x.size(0), -1) # Flatten

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [5]:
# ----------------------------
# Random Seeds and Model Instance (same style)
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(41)
model = VWWConvNet().to(device)

In [6]:
# ----------------------------
# Loss & Optimizer (same style)
# ----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# ----------------------------
# Training (same style)
# ----------------------------
def train(model, loader, criterion, optimizer, device):
    model.train() # training mode

    correct = 0
    total = 0
    running_loss = 0

    for X, y in loader:
        X = X.to(device)
        y = y.to(device)

        # Forward
        outputs = model(X)

        # Loss
        loss = criterion(outputs, y)

        # Backward
        optimizer.zero_grad(set_to_none = True)
        loss.backward()
        optimizer.step()

        # Statistics
        batch_size = y.size(0)
        running_loss += loss.item() * batch_size
        preds = outputs.argmax(dim = 1)
        correct += (preds == y).sum().item()
        total += batch_size

    avg_loss = running_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

In [8]:
# ----------------------------
# Testing (same style)
# ----------------------------
def test(model, loader, criterion, device):
    model.eval() # testing mode

    correct = 0
    total = 0
    running_loss = 0

    with torch.no_grad():
        for X, y in loader:
            X = X.to(device)
            y = y.to(device)

            # Forward
            outputs = model(X)

            # Loss
            loss = criterion(outputs, y)

            # Statistics
            batch_size = y.size(0)
            running_loss += loss.item() * batch_size
            preds = outputs.argmax(dim = 1)
            correct += (preds == y).sum().item()
            total += batch_size

    avg_loss = running_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

In [9]:
# ----------------------------
# Run (same style print)
# ----------------------------
epochs = 20
start_time = time.time()

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    print(
        f"Epoch: {epoch}/{epochs} | "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc * 100:.2f}% | "
        f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc * 100:.2f}%"
    )

print(f"Training Time: {(time.time() - start_time) / 60} minutes!")

Epoch: 1/20 | Train Loss: 0.7246 | Train Acc: 59.94% | Test Loss: 0.6748 | Test Acc: 59.80%
Epoch: 2/20 | Train Loss: 0.6319 | Train Acc: 64.90% | Test Loss: 0.6172 | Test Acc: 67.40%
Epoch: 3/20 | Train Loss: 0.6032 | Train Acc: 67.94% | Test Loss: 0.7196 | Test Acc: 57.90%
Epoch: 4/20 | Train Loss: 0.5862 | Train Acc: 69.31% | Test Loss: 0.5890 | Test Acc: 67.40%
Epoch: 5/20 | Train Loss: 0.5685 | Train Acc: 70.81% | Test Loss: 0.5815 | Test Acc: 68.00%
Epoch: 6/20 | Train Loss: 0.5723 | Train Acc: 70.96% | Test Loss: 0.6417 | Test Acc: 64.30%
Epoch: 7/20 | Train Loss: 0.5568 | Train Acc: 71.96% | Test Loss: 0.5491 | Test Acc: 71.40%
Epoch: 8/20 | Train Loss: 0.5525 | Train Acc: 72.58% | Test Loss: 0.5367 | Test Acc: 73.60%
Epoch: 9/20 | Train Loss: 0.5345 | Train Acc: 73.76% | Test Loss: 0.5477 | Test Acc: 70.90%
Epoch: 10/20 | Train Loss: 0.5270 | Train Acc: 73.78% | Test Loss: 0.5179 | Test Acc: 73.20%
Epoch: 11/20 | Train Loss: 0.5221 | Train Acc: 74.46% | Test Loss: 0.5264 | Tes

In [10]:
# ----------------------------
# Save to Drive (same style)
# ----------------------------
from google.colab import drive
drive.mount('/content/drive')

torch.save(model.state_dict(), "/content/drive/My Drive/Colab Notebooks/stm_vww_model.pth")
print("✅ Model saved as stm_vww_model.pth")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved as stm_vww_model.pth
